# Problem 4
For this problem, you need to use the built-in sklearn digits dataset. You can load this data using Sklearn.datasets.load_digits (*, n_class=10,return_X_y=False, as_frame=False) Divide the data into training and test sets using train_test_split and random_state=0 The goal is to train a Random Forest classifier and optimize its performance on this data.
1. Identify the most important parameters that affect the performance of the Random Forest classifier and outline your experimental design (using 4-fold cross validation) to learn the optimal values for these parameters.
2. Analyze the results of the classifier using its optimal parameters and comment on its generalization capability.
3. Visualize and explain the relevant features identified by the Random Forest classifier.
+ Create a white 8x8 image that represents the original 64 features. Map each identified relevant feature to this 2D image and display it using a grey scale that reflects its importance (e.g. 0 most relevant feature and 255  least relevant feature).
4. Identify one misclassified sample from each class (if they exist). Visualize each misclassified sample as an 8x8 image, and use its nearest neighbors and the learned important features to explain why it was misclassified.

Hint: for examples on how to read this data and visualize it, check
https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glrauto-examples-classification-plot-digits-classification-py

In [ ]:
import sklearn.datasets

sklearn.datasets.load_digits(*, n_class=10, return_X_y=False, frame=False)